In [2]:
import pandas as pd
import numpy as np
import datetime as dt # for date data type
import seaborn as sns # For creating plots
import geopandas as gdp
import matplotlib.pyplot as plt
import plotly.express as px
import holoviews as hv

import numexpr
from platform import python_version

import panel as pn

pn.extension()

print(python_version())

ImportError: cannot import name 'Box' from 'bokeh.models' (c:\Users\Merlin\anaconda3\lib\site-packages\bokeh\models\__init__.py)

^C

Note: you may need to restart the kernel to use updated packages.


In [20]:
df= pd.read_csv("df_clean.csv")
df.head(5)

,Unnamed: 0,Juhtumi nr,Toimumisaeg,Isikuid,Hukkunuid,Vigastatuid,Sõidukeid,Aadress (PPA),Maja nr (PPA),Tänav (PPA),...,Tee km (PPA),Ilmastik [1],Valgustus [1],Valgustus [2],GPS X,GPS Y,Toimumisaeg2,Asukoht,lon,lat
0,0,2000140000057,24.10.2014 08:45,2.0,0,1,1.0,Harju maakond Tallinn Kesklinna linnaosa Esto...,9,Estonia pst,...,NaN,Selged olud,Valge aeg,Valge aeg,6588678,542647,2014-10-24 08:45:00,POINT (24.7514787193144 59.43379595290958),24.751479,59.433796
1,1,2000140000067,24.10.2014 13:45,2.0,0,1,2.0,Harju maakond Tallinn Põhja-Tallinna linnaosa...,NaN,Kopli tn,...,NaN,Pilvised olud,Valge aeg,Valge aeg,6589522,541467,2014-10-24 13:45:00,POINT (24.73084975449409 59.44148846858757),24.730850,59.441488
2,2,2000140000123,8/11/14,2.0,0,1,2.0,Harju maakond Tallinn Pirita linnaosa Meriväl...,NaN,Merivälja tee,...,NaN,Pilvised olud,Valge aeg,Valge aeg,6593961,547646,2014-08-11 00:00:00,POINT (24.84072360549811 59.48068117988471),24.840724,59.480681
3,3,2000140000235,17.11.2014 17:32,2.0,0,2,2.0,Harju maakond Keila vald,NaN,KEILA - HAAPSALU,...,"10,95",Pilvised olud,Pimeda aeg,Pimeda ajal valgustus ei põle,6569324,516628,2014-11-17 17:32:00,POINT (24.29152856115738 59.26188409298865),24.291529,59.261884
4,4,2000150000442,28.04.2015 07:55,2.0,0,1,2.0,Harju maakond Tallinn Kesklinna linnaosa Pärn...,123a,Pärnu mnt,...,NaN,Vihmasadu,Valge aeg,Valge aeg,6586430,541953,2015-04-28 07:55:00,POINT (24.73881452757407 59.41368770550172),24.738815,59.413688


In [23]:
fig = px.scatter_mapbox(df, lat ="lat", lon="lon", height=600, zoom=6.5)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [40]:
min = pd.to_datetime(df.Toimumisaeg.min())
max =pd.to_datetime(df.Toimumisaeg.max())

# Create slider for time series
datetime_range_slider = pn.widgets.DatetimeRangeSlider(
    name='Datetime Range Slider',
    start=min, end=max,
    value=(min, max),
    step=10000
)

datetime_range_slider

ImportError: cannot import name 'warn' from 'bokeh.util.warnings' (c:\Users\Merlin\anaconda3\lib\site-packages\bokeh\util\warnings.py)

DatetimeRangeSlider(end=Timestamp('2021-09-09 0..., name='Datetime Range Slider', start=Timestamp('2012-01-01 0..., step=10000, value=(Timestamp('2012-01-01 01:..., value_end=Timestamp('2021-09-09 0..., value_start=Timestamp('2012-01-01 0...)

In [ ]:
# Function to display current year
@pn.depends(DatetimeRangeSlider.param.value_throttled)
def tiitel(datetime_range_slider):
    return '### Liiklusõnnetuste statistika ajavahemikul {}'.format(DatetimeRangeSlider)

  Using cached bokeh-3.1.1-py3-none-any.whl (8.3 MB)
  Attempting uninstall: bokeh
    Found existing installation: bokeh 2.4.3
    Uninstalling bokeh-2.4.3:
      Successfully uninstalled bokeh-2.4.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
panel 0.14.0 requires bokeh<2.5.0,>=2.4.0, but you have bokeh 3.1.1 which is incompatible.
geoviews 1.9.6 requires bokeh<2.5,>=2.4.0, but you have bokeh 3.1.1 which is incompatible.


ERROR: Invalid requirement: 'panel==0.14.4,'


In [ ]:
dashboard = pn.WidgetBox(pn.Column(pn.Row(year_selected, datetime_range_slider),
                         pn.Row(pn.bind(plot_map, year)), align="start",
                         sizing_mode="stretch_width"))

https://towardsdatascience.com/using-panel-and-hvplot-to-visualize-decades-of-wildfires-2ac8d6f5db25
https://hvplot.holoviz.org/user_guide/Widgets.html
https://panel.holoviz.org/reference/widgets/DatetimeRangeSlider.html

     ---------------------------------------- 18.5/18.5 MB 4.9 MB/s eta 0:00:00
  Attempting uninstall: bokeh
    Found existing installation: bokeh 3.1.1
    Uninstalling bokeh-3.1.1:
      Successfully uninstalled bokeh-3.1.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-bokeh 3.0.7 requires bokeh==3.*, but you have bokeh 2.4.3 which is incompatible.
